In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker
import datetime as datetime

In [2]:
db_url = 'sqlite:///datalake.sqlite'
engine = create_engine(db_url, echo=True)
# Create a session
Session = sessionmaker(bind=engine)
session = Session()

# Replace 'your_tournament_id' with the actual tournament ID
tournament_id = 110413046183015975

# Raw SQL query to retrieve all frames for a specific tournament
sql_query = text(
    """
    SELECT matches.*, frames_two.*
    FROM frames_two
    JOIN games ON frames_two.game_id = games.game_id
    JOIN matches ON games.match_id = matches.match_id
    WHERE matches.tournament_id = :tournament_id
    """
)

# Get a connection from the session
connection = session.connection()

# Execute the query
result = connection.execute(sql_query, {"tournament_id": tournament_id})
df = pd.DataFrame(result.fetchall(), columns=result.keys())
df

2024-01-15 13:24:36,857 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-15 13:24:36,858 INFO sqlalchemy.engine.Engine 
    SELECT matches.*, frames_two.*
    FROM frames_two
    JOIN games ON frames_two.game_id = games.game_id
    JOIN matches ON games.match_id = matches.match_id
    WHERE matches.tournament_id = ?
    
2024-01-15 13:24:36,859 INFO sqlalchemy.engine.Engine [generated in 0.00055s] (110413046183015975,)


,match_id,start_time,tournament_id,strategy,name_team0,name_team1,result_team0,result_team1,id,game_time,...,participant9_currentHealth,participant9_maxHealth,participant10_totalGold,participant10_level,participant10_kills,participant10_deaths,participant10_assists,participant10_creepScore,participant10_currentHealth,participant10_maxHealth
0,110413046183474732,2023-06-10 16:00:00.000000,110413046183015975,bestOf1,paiN Gaming,LOUD,1,0,1,2023-06-10 16:32:45.557000,...,0,0,0,1,0,0,0,0,0,0
1,110413046183474732,2023-06-10 16:00:00.000000,110413046183015975,bestOf1,paiN Gaming,LOUD,1,0,2,2023-06-10 16:32:45.557000,...,0,0,0,1,0,0,0,0,0,0
2,110413046183474732,2023-06-10 16:00:00.000000,110413046183015975,bestOf1,paiN Gaming,LOUD,1,0,3,2023-06-10 16:32:45.557000,...,0,0,0,1,0,0,0,0,0,0
3,110413046183474732,2023-06-10 16:00:00.000000,110413046183015975,bestOf1,paiN Gaming,LOUD,1,0,4,2023-06-10 16:32:47.754000,...,0,0,0,1,0,0,0,0,0,0
4,110413046183474732,2023-06-10 16:00:00.000000,110413046183015975,bestOf1,paiN Gaming,LOUD,1,0,5,2023-06-10 16:32:47.787000,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859292,110471059652887260,2023-09-09 16:00:00.000000,110413046183015975,bestOf5,paiN Gaming,LOUD,1,3,859293,2023-09-09 20:00:25.195000,...,2020,2022,5752,10,0,1,11,28,131,2213
859293,110471059652887260,2023-09-09 16:00:00.000000,110413046183015975,bestOf5,paiN Gaming,LOUD,1,3,859294,2023-09-09 20:00:26.198000,...,2022,2022,5754,10,0,1,11,28,137,2213
859294,110471059652887260,2023-09-09 16:00:00.000000,110413046183015975,bestOf5,paiN Gaming,LOUD,1,3,859295,2023-09-09 20:00:27.196000,...,2022,2022,5757,10,0,1,11,28,143,2213
859295,110471059652887260,2023-09-09 16:00:00.000000,110413046183015975,bestOf5,paiN Gaming,LOUD,1,3,859296,2023-09-09 20:00:27.299000,...,2022,2022,5758,10,0,1,11,28,146,2213


In [3]:
df[['result_team0', 'result_team1']]

,result_team0,result_team1
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0
...,...,...
859292,1,3
859293,1,3
859294,1,3
859295,1,3


# Create Target Column

In [4]:
def who_won(row):
    if row['result_team0'] == 1 and row['result_team1'] == 0:
        return 0
    if row['result_team0'] == 0 and row['result_team1'] == 1:
        return 1
    else:
        return 3

In [5]:
df['target'] = df.apply(who_won, axis=1)

In [6]:
df[['name_team0', 'name_team1', 'result_team0', 'result_team1', 'target']]

,name_team0,name_team1,result_team0,result_team1,target
0,paiN Gaming,LOUD,1,0,0
1,paiN Gaming,LOUD,1,0,0
2,paiN Gaming,LOUD,1,0,0
3,paiN Gaming,LOUD,1,0,0
4,paiN Gaming,LOUD,1,0,0
...,...,...,...,...,...
859292,paiN Gaming,LOUD,1,3,3
859293,paiN Gaming,LOUD,1,3,3
859294,paiN Gaming,LOUD,1,3,3
859295,paiN Gaming,LOUD,1,3,3


In [7]:
df['target'].value_counts()

target
0    314573
1    313543
3    231181
Name: count, dtype: int64

In [8]:
df_playoffs = df[df['target'].isin([3])]
df = df[df['target'].isin([0, 1])]

In [9]:
df

,match_id,start_time,tournament_id,strategy,name_team0,name_team1,result_team0,result_team1,id,game_time,...,participant9_maxHealth,participant10_totalGold,participant10_level,participant10_kills,participant10_deaths,participant10_assists,participant10_creepScore,participant10_currentHealth,participant10_maxHealth,target
0,110413046183474732,2023-06-10 16:00:00.000000,110413046183015975,bestOf1,paiN Gaming,LOUD,1,0,1,2023-06-10 16:32:45.557000,...,0,0,1,0,0,0,0,0,0,0
1,110413046183474732,2023-06-10 16:00:00.000000,110413046183015975,bestOf1,paiN Gaming,LOUD,1,0,2,2023-06-10 16:32:45.557000,...,0,0,1,0,0,0,0,0,0,0
2,110413046183474732,2023-06-10 16:00:00.000000,110413046183015975,bestOf1,paiN Gaming,LOUD,1,0,3,2023-06-10 16:32:45.557000,...,0,0,1,0,0,0,0,0,0,0
3,110413046183474732,2023-06-10 16:00:00.000000,110413046183015975,bestOf1,paiN Gaming,LOUD,1,0,4,2023-06-10 16:32:47.754000,...,0,0,1,0,0,0,0,0,0,0
4,110413046183474732,2023-06-10 16:00:00.000000,110413046183015975,bestOf1,paiN Gaming,LOUD,1,0,5,2023-06-10 16:32:47.787000,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628111,110413046183474846,2023-08-06 20:00:00.000000,110413046183015975,bestOf1,Vivo Keyd Stars,RED Kalunga,1,0,628112,2023-08-06 20:50:29.103000,...,2533,6079,10,0,5,3,30,2213,2213,0
628112,110413046183474846,2023-08-06 20:00:00.000000,110413046183015975,bestOf1,Vivo Keyd Stars,RED Kalunga,1,0,628113,2023-08-06 20:50:30.101000,...,2533,6082,10,0,5,3,30,2213,2213,0
628113,110413046183474846,2023-08-06 20:00:00.000000,110413046183015975,bestOf1,Vivo Keyd Stars,RED Kalunga,1,0,628114,2023-08-06 20:50:31.103000,...,2533,6084,10,0,5,3,30,2213,2213,0
628114,110413046183474846,2023-08-06 20:00:00.000000,110413046183015975,bestOf1,Vivo Keyd Stars,RED Kalunga,1,0,628115,2023-08-06 20:50:31.374000,...,2533,6085,10,0,5,3,30,2213,2213,0


# NEXT STEPS

* XGBoost install
* Test Deep Learning (PyTorch With CUDA?) pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
* Create a column called Game_time, where is the specific time, first frame tive vs last frame time, get the total, and put  in each row the specific time. Ex: 00:30 (first 30 seconds), 11:00 (first 11 minutes)
* Random Forest ta bom, cria logo uma pipeline
* About the dragons, dragons types change every update

# GAME_TIME

In [51]:
df['time'] = pd.to_datetime(df['game_time'])

/tmp/ipykernel_682/283679967.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time'] = pd.to_datetime(df['game_time'])


In [52]:
df['game_id']

0         110413046183474733
1         110413046183474733
2         110413046183474733
3         110413046183474733
4         110413046183474733
                 ...        
628111    110413046183474847
628112    110413046183474847
628113    110413046183474847
628114    110413046183474847
628115    110413046183474847
Name: game_id, Length: 628116, dtype: int64

In [55]:
df['time_'] =  df.groupby('game_id')['time'].apply(lambda x: (x - x.iloc[0])).reset_index(drop=True).dt.total_seconds()

/tmp/ipykernel_682/2997544533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_'] =  df.groupby('game_id')['time'].apply(lambda x: (x - x.iloc[0])).reset_index(drop=True).dt.total_seconds()


In [58]:
df.groupby('game_id').get_group(110413046183474733)[['time_','game_time']]

,time_,game_time
0,0.000,2023-06-10 16:32:45.557000
1,0.000,2023-06-10 16:32:45.557000
2,0.000,2023-06-10 16:32:45.557000
3,1.363,2023-06-10 16:32:47.754000
4,1.498,2023-06-10 16:32:47.787000
...,...,...
6764,2017.061,2023-06-10 17:06:52.889000
6765,2017.161,2023-06-10 17:06:53.887000
6766,2017.794,2023-06-10 17:06:54.887000
6767,2017.800,2023-06-10 17:06:55.058000


# Features

* Farm Diff

# TRAINING

In [65]:
df.columns[9:]

Index(['game_time', 'game_id', 'blueTeam_totalGold', 'blueTeam_inhibitors',
       'blueTeam_towers', 'blueTeam_barons', 'blueTeam_totalKills',
       'blueTeam_dragons', 'redTeam_totalGold', 'redTeam_inhibitors',
       'redTeam_towers', 'redTeam_barons', 'redTeam_totalKills',
       'redTeam_dragons', 'participant1_totalGold', 'participant1_level',
       'participant1_kills', 'participant1_deaths', 'participant1_assists',
       'participant1_creepScore', 'participant1_currentHealth',
       'participant1_maxHealth', 'participant2_totalGold',
       'participant2_level', 'participant2_kills', 'participant2_deaths',
       'participant2_assists', 'participant2_creepScore',
       'participant2_currentHealth', 'participant2_maxHealth',
       'participant3_totalGold', 'participant3_level', 'participant3_kills',
       'participant3_deaths', 'participant3_assists',
       'participant3_creepScore', 'participant3_currentHealth',
       'participant3_maxHealth', 'participant4_totalGold',

In [119]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import RandomForestClassifier

In [126]:
features = df[['blueTeam_totalGold', 'blueTeam_inhibitors',
       'blueTeam_towers', 'blueTeam_barons', 'blueTeam_totalKills',
 'redTeam_totalGold', 'redTeam_inhibitors',
       'redTeam_towers', 'redTeam_barons', 'redTeam_totalKills',
 'participant1_totalGold', 'participant1_level',
       'participant1_kills', 'participant1_deaths', 'participant1_assists',
       'participant1_creepScore', 'participant1_currentHealth',
       'participant1_maxHealth', 'participant2_totalGold',
       'participant2_level', 'participant2_kills', 'participant2_deaths',
       'participant2_assists', 'participant2_creepScore',
       'participant2_currentHealth', 'participant2_maxHealth',
       'participant3_totalGold', 'participant3_level', 'participant3_kills',
       'participant3_deaths', 'participant3_assists',
       'participant3_creepScore', 'participant3_currentHealth',
       'participant3_maxHealth', 'participant4_totalGold',
       'participant4_level', 'participant4_kills', 'participant4_deaths',
       'participant4_assists', 'participant4_creepScore',
       'participant4_currentHealth', 'participant4_maxHealth',
       'participant5_totalGold', 'participant5_level', 'participant5_kills',
       'participant5_deaths', 'participant5_assists',
       'participant5_creepScore', 'participant5_currentHealth',
       'participant5_maxHealth', 'participant6_totalGold',
       'participant6_level', 'participant6_kills', 'participant6_deaths',
       'participant6_assists', 'participant6_creepScore',
       'participant6_currentHealth', 'participant6_maxHealth',
       'participant7_totalGold', 'participant7_level', 'participant7_kills',
       'participant7_deaths', 'participant7_assists',
       'participant7_creepScore', 'participant7_currentHealth',
       'participant7_maxHealth', 'participant8_totalGold',
       'participant8_level', 'participant8_kills', 'participant8_deaths',
       'participant8_assists', 'participant8_creepScore',
       'participant8_currentHealth', 'participant8_maxHealth',
       'participant9_totalGold', 'participant9_level', 'participant9_kills',
       'participant9_deaths', 'participant9_assists',
       'participant9_creepScore', 'participant9_currentHealth',
       'participant9_maxHealth', 'participant10_totalGold',
       'participant10_level', 'participant10_kills', 'participant10_deaths',
       'participant10_assists', 'participant10_creepScore',
       'participant10_currentHealth', 'participant10_maxHealth', 'time_']]
target = df['target']

In [127]:
X_train, X_test, y_train, y_test = train_test_split(
     features, target, test_size=0.25, random_state=42)

In [122]:
# Normalizer
# transformer = Normalizer().fit(X_train) 
# X_train = transformer.transform(X_train)
# X_test = transformer.transform(X_test)

In [130]:
%%time
clf = RandomForestClassifier(max_depth=18, random_state=0)
model = clf.fit(X_train, y_train)
y_pred = model.predict(X_test)

CPU times: user 1min 54s, sys: 83.1 ms, total: 1min 55s
Wall time: 1min 55s


In [132]:
X_test

,blueTeam_totalGold,blueTeam_inhibitors,blueTeam_towers,blueTeam_barons,blueTeam_totalKills,redTeam_totalGold,redTeam_inhibitors,redTeam_towers,redTeam_barons,redTeam_totalKills,...,participant9_maxHealth,participant10_totalGold,participant10_level,participant10_kills,participant10_deaths,participant10_assists,participant10_creepScore,participant10_currentHealth,participant10_maxHealth,time_
5430,47958,0,3,1,19,44063,0,3,0,10,...,1749,5893,10,1,1,7,34,1976,1976,1595.234
282755,9845,0,0,0,0,9993,0,0,0,1,...,985,1627,4,0,0,1,9,875,875,282.709
53703,38169,0,3,1,8,34736,0,3,0,6,...,1507,4855,9,0,0,4,35,1588,1588,263.658
592940,11168,0,0,0,3,11266,0,0,0,4,...,1049,1945,4,1,0,2,11,607,1014,1718.689
299230,8248,0,0,0,1,7795,0,0,0,1,...,981,1090,3,0,0,1,4,824,824,942.385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609575,34508,0,3,0,9,26530,0,1,0,1,...,1301,4288,7,0,1,1,10,841,1522,405.460
255594,48266,0,3,0,10,50671,0,3,0,19,...,2023,6411,10,0,3,13,38,0,2014,2076.539
437694,66285,0,5,2,21,62601,0,4,0,12,...,2146,7716,11,1,6,6,27,2388,2388,2447.124
474523,47922,0,6,1,13,34389,0,1,0,4,...,1584,5039,9,0,1,4,44,1877,1877,2182.462


In [131]:
target_names = ['BLUE', 'RED']
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

        BLUE       0.99      0.99      0.99     78260
         RED       0.99      0.99      0.99     78769

    accuracy                           0.99    157029
   macro avg       0.99      0.99      0.99    157029
weighted avg       0.99      0.99      0.99    157029



# Predict Playoffs